# SQL Generation with Transformer API

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [4]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [5]:
print(available_memory)

15828320256


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [6]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [7]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [8]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [9]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [10]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [11]:
prompt_v1 = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Revenue = price * quantity
- Cost = supply_price * quantity

### Database Schema
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY,
   name VARCHAR(50),
   address VARCHAR(100),
   region_id INTEGER
);

CREATE TABLE regions (
   region_id INTEGER PRIMARY KEY,
   name VARCHAR(50)
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY,
  name VARCHAR(50)
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  customer_id INTEGER,
  salesperson_id INTEGER,
  sale_date DATE,
  quantity INTEGER
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  supply_price DECIMAL(10,2)
);

-- customers.region_id = regions.region_id
-- sales.customer_id = customers.customer_id
-- sales.product_id = products.product_id
-- product_suppliers.product_id = products.product_id

### Answer
Here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [12]:
prompt_v2 = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- Use revenue and cost columns directly if available.
- If not available, compute them.
- Avoid hallucinating columns that don't exist.

### Database Schema
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY,
   name VARCHAR(50),
   address VARCHAR(100)
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  region VARCHAR(50)
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  customer_id INTEGER,
  salesperson_id INTEGER,
  sale_date DATE,
  quantity INTEGER,
  revenue DECIMAL(10,2) -- NEW: total revenue already calculated
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  supply_price DECIMAL(10,2)
);

-- sales.product_id = products.product_id
-- sales.customer_id = customers.customer_id
-- sales.salesperson_id = salespeople.salesperson_id
-- product_suppliers.product_id = products.product_id

### Answer
Here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [13]:
prompt_v3 = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- Provide results grouped by product
- Return only the top 5 products with the highest total revenue
- Sort the results by revenue descending
- Revenue = price * quantity

### Database Schema
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY,
   name VARCHAR(50),
   address VARCHAR(100)
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  region VARCHAR(50)
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  customer_id INTEGER,
  salesperson_id INTEGER,
  sale_date DATE,
  quantity INTEGER
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  supply_price DECIMAL(10,2)
);

-- sales.product_id = products.product_id
-- sales.customer_id = customers.customer_id
-- sales.salesperson_id = salespeople.salesperson_id
-- product_suppliers.product_id = products.product_id

### Answer
Here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [14]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()

    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [15]:
question = "What are the top 5 products with the highest revenue in the New York region last month?"

prompt = prompt_v1
print("🔹 Version 1:")
print(generate_query(question))

prompt = prompt_v2
print("\n🔹 Version 2:")
print(generate_query(question))

prompt = prompt_v3
print("\n🔹 Version 3:")
print(generate_query(question))

🔹 Version 1:

SELECT p.name,
       SUM(s.quantity * p.price) AS revenue
FROM products p
JOIN sales s ON p.product_id = s.product_id
JOIN customers c ON s.customer_id = c.customer_id
JOIN regions r ON c.region_id = r.region_id
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
  AND r.name = 'New York'
GROUP BY p.name
ORDER BY revenue DESC NULLS LAST
LIMIT 5;

🔹 Version 2:

SELECT p.name,
       SUM(s.revenue) AS total_revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.name
ORDER BY total_revenue DESC NULLS LAST
LIMIT 5;

🔹 Version 3:

SELECT p.name,
       SUM(s.quantity) AS total_quantity
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.name


# 📝 Comparative Report on Prompt Variations and SQLCoder-7B Output

## 🎯 Objective
The goal of this experiment is to evaluate how modifying the structure and content of prompts affects the performance of the SQLCoder-7B model in generating SQL queries. All prompt versions were tested using the same question to ensure a fair comparison:

**"What are the top 5 products with the highest revenue in the New York region last month?"**

---

## 📦 Prompt Variants:

- **Version 1 (`prompt_v1`)**  
  A new table named `regions` was introduced and linked directly to the `customers` table via a `region_id` column to clearly define the customer-region relationship.

- **Version 2 (`prompt_v2`)**  
  A `revenue` column was added directly to the `sales` table, representing total revenue per sale. The objective was to test whether the model would use the available column instead of computing it manually.

- **Version 3 (`prompt_v3`)**  
  The instructions were updated to request only the **top 5 products** with the highest **total revenue**, grouped by product and sorted in descending order. The schema remained unchanged.

---

## ✅ Results:

- 🔹 **Version 1:**  
  The model correctly used the `regions` table and joined it to customers via `region_id`. It computed revenue using `price * quantity`, grouped results by product, and applied sorting and a limit of 5. The output was accurate and aligned with the query goal.

- 🔹 **Version 2:**  
  The model directly utilized the `revenue` column as instructed, avoiding manual calculations. It filtered using `salespeople.region`, which is a logical interpretation of the schema. The result was accurate and efficient.

- 🔹 **Version 3:**  
  The model incorrectly summed `quantity` instead of calculating or using revenue (`price * quantity`). This indicates that the model either misunderstood or ignored the revenue-focused instruction, leading to an inaccurate result.

---

## 🧠 Observations & Insights:

- Explicitly defining relationships in the schema (as in Version 1) significantly improves the model's understanding and reduces hallucinations.
- Providing pre-computed columns like `revenue` helps the model produce cleaner and more accurate queries by reducing complexity.
- When multiple instructions are provided (as in Version 3), the model may miss or ignore some of them, especially if they’re not well aligned with the schema structure.

---

## ✅ Recommendations:

- Always **clarify relationships explicitly** in the schema, even if they seem implied.
- If a metric is critical (like revenue), consider including it directly in the schema to guide the model.
- Don't rely solely on written instructions—support them with schema design that reinforces the task requirements.

---
